In [ ]:
%cd ..

In [ ]:
from os import getenv

from uuid import uuid4

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.messages import HumanMessage, AIMessage, SystemMessage

from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os
print(f"current working directory is: {os.getcwd()}")

import sys
if "./src" not in sys.path: sys.path.append("./src")

print (sys.path)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

from langgraph.types import Command

from smart_investigator.foundation.agents.ticketing_agent_graph import create_ticketing_agent
from smart_investigator.foundation.agents.travel_agent_graph import create_travel_agent
from smart_investigator.foundation.agents.travel_agent_tool_agents import create_ticketing_agent_tool

In [ ]:
ticketing_llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

user_inputs = [
    "I want to go to Bali, Indonesia. 2 tickets.",
    "Sydney, Australia. Round-trip please. ",
    "yes. Proceed and invpoice me",
    "...",
    "exit",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

greeting = "Hi! I'm your ticketing assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent_tkt = create_ticketing_agent(ticketing_llm, InMemorySaver())
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent_tkt.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        response = agent_tkt.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()

In [ ]:
user_inputs = [
    "I want to go to Bali. For 2 adults. Next week Wednesday.",
    "Departing from Sydney Australia, 30th November return. Nothing else. ",
    "yes, proceed with the booking",
    "ok,",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

llm_travel = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

greeting = "Hi! I'm your travel assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent = create_travel_agent(llm_travel, InMemorySaver(), agent_tools=[create_ticketing_agent_tool(agent_tkt=agent_tkt)])
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit):") # next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        response = agent.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()